In [ ]:
%%bash
pip install llama-cpp-python flask-ngrok

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
ngrok_api_key = os.getenv("NGROK_API_KEY")

In [ ]:
!ngrok authtoken ngrok_api_key

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen2-0.5B-Instruct-GGUF",
    filename="*q8_0.gguf",
    verbose=False,
    n_ctx=2048,
#     n_gpu_layers=-1
)


prompt = """
You are a helpful Doctor, Help the patient below with their queries and provide a detail ansewr to them
Patient: Why DOes my nose hurt so bad Doctor, its like there is severe suffocation and my thorat is also hurting, Can you devise a full Weekly prescription as to what i shoudl take and also tell me safety
precautions to take, please, i would be forever thankful?
Doctor:
"""


output = llm(
      prompt, # Prompt
      max_tokens=None, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"],# Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
)



print(output['choices'][0]['text'])

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route(“/”)
def hello():
    return “Hello World!! from Google Colab”

if __name__ == ‘__main__’:
app.run()

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from llama_cpp import Llama

app = Flask(__name__)
run_with_ngrok(app)
# Load the model
checkpoint = "Qwen/Qwen2-0.5B-Instruct-GGUF"
filename = "*q8_0.gguf"
llm = Llama.from_pretrained(
    repo_id=checkpoint,
    filename=filename,
    verbose=False,
    n_ctx=2048,
#     n_gpu_layers=-1
)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prompt = data['prompt']
    max_tokens = data.get('max_tokens', 32)

    # Generate text
    response = llm(prompt,
                   max_tokens=max_tokens,
                   stop=["Q:", "\n"],
                   echo=True)

    return jsonify(response)

if __name__ == '__main__':
    app.run()


In [ ]:
from llama_cpp import Llama
from llama_cpp.llama_chat_format import MoondreamChatHandler

chat_handler = MoondreamChatHandler.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*mmproj*",
)

llm = Llama.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*text-model*",
  chat_handler=chat_handler,
  n_ctx=2048, # n_ctx should be increased to accommodate the image embedding
)

response = llm.create_chat_completion(
    messages = [
        {
            "role": "user",
            "content": [
                {"type" : "text", "text": "What's in this image?"},
                {"type": "image_url", "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" } }

            ]
        }
    ]
)
print(response["choices"][0]["text"])